<center>

# **Árboles y Clasificación Bayesiana: EJEMPLO 2:**
Colab: https://colab.research.google.com/drive/1K9iPTCq1O2OVCIL0yMw34OXmOG7W7hPK?usp=sharing
---
Introducción a la Inteligencia Artificial, Código: 3010476

Inteligencia Artificial, Código: 3007855

---
**Monitores:**

Maria Alejandra Muñoz Alarcón

---

Denilson Andres Molina Truyot 

---
</center>

In [ ]:
# Cargar librerias
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sb
 
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')

from sklearn.model_selection import StratifiedKFold
# librerías Árboles de Decisión
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont

# librerías Gaussian Naive Bayes
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import make_scorer

# CONTEXTO
Las columnas que tenemos son:

- *ingresos*: los ingresos mensuales de la familia
- *gastos comunes*: pagos de luz, agua, gas, etc mensual
- *pago coche*: si se está pagando cuota por uno o más coches, y los gastos en combustible, etc al mes.
- *gastos_otros*: compra en supermercado y lo necesario para vivir al mes
- *ahorros*: suma de ahorros dispuestos para usar en la compra de la casa.
- *vivienda*: precio de la vivienda que quiere comprar esa familia
- *estado civil*:
  0. soltero
  1. casados
  2. divorciados
- *hijos*: cantidad de hijos menores y que no trabajan.
- *trabajo*:
  0. sin empleo 
  1. autónomo (freelance)
  2. empleado
  3. empresario
  4. pareja: autónomos
  5. pareja: empleados
  6. pareja: autónomo y asalariado
  7. pareja:empresario y autónomo
  8. pareja: empresarios los dos o empresario y empleado
- comprar: (esta será nuestra columna de salida)
  0. Alquilar 
  1. Comprar 

**Algunos supuestos para el problema formulado:**
- Está pensado en Euros pero podría ser cualquier otra moneda
- No tiene en cuenta ubicación geográfica (en la realidad esto afecta bastante el precio)
- Se supone una hipoteca fija a 30 años con interés de mercado «bajo».

In [ ]:
#Cargar datos.
# Carga de datos o teniendo el csv en la carpeta nube del colab.
#dataframe = pd.read_csv('comprar_alquilar.csv')
#dataframe.head()

In [ ]:
#Para abrir desde colab sin tenerlo en la nube.
from google.colab import files
import io
uploaded = files.upload()
dataframe = pd.read_csv(io.BytesIO(uploaded['comprar_alquilar.csv']))

## 1. Estadística descriptiva

*Sug: Realizar todos los cambios que sean necesarios para los datos y luego realizar el análisis exploratorio de Variable Objetivo vs Atributos depurados.*

In [ ]:
print(dataframe.groupby('comprar').size())
dataframe.shape

In [ ]:
dataframe.drop(['comprar'], axis=1).hist()
plt.show()

### Cálculos financieros
Procesando algunas de estas columnas podríamos agrupar los gastos. 

También crear una columna llamada financiar que será la resta del precio de la vivienda con los ahorros de la familia.

In [ ]:
dataframe['gastos']=(dataframe['gastos_comunes']+dataframe['gastos_otros']+dataframe['pago_coche'])
dataframe['financiar']=dataframe['vivienda']-dataframe['ahorros']
dataframe.drop(['gastos_comunes','gastos_otros','pago_coche'], axis=1).head(5)

Ahora usando el `describe()` de pandas podemos ver los estadisticos de los datos.

In [ ]:
reduced = dataframe.drop(['gastos_comunes','gastos_otros','pago_coche','estado_civil','hijos','trabajo','comprar'], axis=1)
reduced.describe()

### Variable objetivo vs Datos involucrados

#### Usando `pairplot`

In [ ]:
#Visualizar relación si existe relación entre las variables que no son la variable objetivo
sb.pairplot(dataframe.dropna(),
            height=4, 
            vars=['comprar','ingresos'],
            kind='scatter')
plt.show()

In [ ]:
#Visualizar relación si existe relación entre las variables que no son la variable objetivo
sb.pairplot(dataframe.dropna(),
            height=4, 
            vars=['comprar','gastos_comunes'],
            kind='scatter')
plt.show()

In [ ]:
#Visualizar relación si existe relación entre las variables que no son la variable objetivo
sb.pairplot(dataframe.dropna(),
            height=4, 
            vars=['comprar','hijos'],
            kind='scatter')
plt.show()

#### Otras formas. Útil para variables categóricas.

In [ ]:
g = sb.catplot(x='hijos', data=dataframe, hue='comprar', kind="count")

g.fig.suptitle("Comprar-Alquilar vs Hijos")


In [ ]:
g = sb.catplot(x='estado_civil', data=dataframe, hue='comprar', kind="count")

g.fig.suptitle("Comprar-Alquilar vs estado_civil")


## 2. Característica escogida para el nodo raíz. ¿Entropia o gini?

In [ ]:
#Característica objetivo: Comprar

#Mezclar datos
dataframe=dataframe.sample(frac=1).reset_index(drop=True)

#Todos los datos excepto la caracteristica comprar
X = dataframe.drop(['comprar'], axis=1)

#CARACTERÍSTICA OBJETIVO
y = dataframe['comprar'] 

#¿Cuáles serían las características/atributos/variables que puede ser posibles candidatos al nodo raíz? Se debe analizar una hipótesis con el análisis exploratorio

X = np.array(X)
y = np.array(y)

### Entropía

In [ ]:
cv = StratifiedKFold(n_splits=6, shuffle=True, random_state=1) 
accuracies = list()
max_attributes = len(list(dataframe))
depth_range = range(1, max_attributes + 1)

# Testearemos la profundidad de 1 a cantidad de atributos +1
for depth in depth_range:
    fold_accuracy = []
    tree_model = tree.DecisionTreeClassifier(criterion='entropy',
                                             min_samples_split=20,
                                             min_samples_leaf=5,
                                             max_depth = depth,
                                             class_weight='balanced') 
    for train_index, valid_index in cv.split(X, y):
        X_train, X_test = X[train_index], X[valid_index]
        y_train, y_test = y[train_index], y[valid_index]

        model = tree_model.fit(X_train, y_train)
        valid_acc = model.score(X_test, y_test) # calculamos la precision con el segmento de validacion
        fold_accuracy.append(valid_acc)

    # almacenamos acc promedio para cada profundidad
    avg = sum(fold_accuracy)/len(fold_accuracy)
    accuracies.append(avg)
    
# Mostramos los resultados obtenidos
df = pd.DataFrame({"Max Depth": depth_range, "Average Accuracy": accuracies})
df = df[["Max Depth", "Average Accuracy"]]
print(df.to_string(index=False))

### Gini

In [ ]:
cv = StratifiedKFold(n_splits=6, shuffle=True, random_state=1) 
accuracies = list()
max_attributes = len(list(dataframe))
depth_range = range(1, max_attributes + 1)

# Testearemos la profundidad de 1 a cantidad de atributos +1
for depth in depth_range:
    fold_accuracy = []
    tree_model = tree.DecisionTreeClassifier(criterion='gini',
                                             min_samples_split=20,
                                             min_samples_leaf=5,
                                             max_depth = depth,
                                             class_weight='balanced') 
    for train_index, valid_index in cv.split(X, y):
        X_train, X_test = X[train_index], X[valid_index]
        y_train, y_test = y[train_index], y[valid_index]

        model = tree_model.fit(X_train, y_train)
        valid_acc = model.score(X_test, y_test) # calculamos la precision con el segmento de validacion
        fold_accuracy.append(valid_acc)

    # almacenamos acc promedio para cada profundidad
    avg = sum(fold_accuracy)/len(fold_accuracy)
    accuracies.append(avg)
    
# Mostramos los resultados obtenidos
df = pd.DataFrame({"Max Depth": depth_range, "Average Accuracy": accuracies})
df = df[["Max Depth", "Average Accuracy"]]
print(df.to_string(index=False))

## 3. Datos de entrenamiento y prueba.

*Sug: Se crean árboles sin testear parámetros solo para tener una vista de los valores de exactitud variando el porcentaje de datos de entrenamiento y prueba. Se debe especificar que estos NO son los árboles definitivos.*

### 10/90

In [ ]:
#Todos los datos excepto la caracteristica comprar
X = dataframe.drop(['comprar'], axis=1)

#Atributo OBJETIVO
y = dataframe['comprar'] 

# Dividimos los datos para entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.1, #90% datos de entrenamiento, 10% prueba
                                                    stratify = y,
                                                    random_state = 1)
# Datos de entrenaminto
conteo = X_train.copy()
conteo['label'] = y_train
conteo.groupby('label').size()

In [ ]:
decision_tree = tree.DecisionTreeClassifier(criterion='gini',
                                             min_samples_split=20,
                                             min_samples_leaf=5,
                                             max_depth = depth,
                                             class_weight='balanced') 

# Ajustamos el modelo con los datos
decision_tree.fit(X_train, y_train)
y_pred_train = decision_tree.predict(X_train)

#Porcentaje de Exactitud con los datos de entrenamiento
acc_train = accuracy_score(y_train, y_pred_train)
print("Exactitud con datos de entrenamiento: {:.2f}".format(acc_train))

y_pred_test = decision_tree.predict(X_test)

#Porcentaje de Exactitud con pruebas
acc_test = accuracy_score(y_test, y_pred_test)
print("Exactitud con datos de pruebas: {:.2f}".format(acc_test))

### 20/80

In [ ]:
#Todos los datos excepto la caracteristica comprar
X = dataframe.drop(['comprar'], axis=1)

#Atributo OBJETIVO
y = dataframe['comprar'] 

# Dividimos los datos para entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.2, #80% datos de entrenamiento, 20% prueba
                                                    stratify = y,
                                                    random_state = 1)
# Datos de entrenaminto
conteo = X_train.copy()
conteo['label'] = y_train
conteo.groupby('label').size()

In [ ]:
decision_tree = tree.DecisionTreeClassifier(criterion='gini',
                                             min_samples_split=20,
                                             min_samples_leaf=5,
                                             max_depth = depth,
                                             class_weight='balanced') 

# Ajustamos el modelo con los datos
decision_tree.fit(X_train, y_train)
y_pred_train = decision_tree.predict(X_train)

#Porcentaje de Exactitud con los datos de entrenamiento
acc_train = accuracy_score(y_train, y_pred_train)
print("Exactitud con datos de entrenamiento: {:.2f}".format(acc_train))

y_pred_test = decision_tree.predict(X_test)

#Porcentaje de Exactitud con pruebas
acc_test = accuracy_score(y_test, y_pred_test)
print("Exactitud con datos de pruebas: {:.2f}".format(acc_test))

### 30/80

In [ ]:
#Todos los datos excepto la caracteristica comprar
X = dataframe.drop(['comprar'], axis=1)

#Atributo OBJETIVO
y = dataframe['comprar'] 

# Dividimos los datos para entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3, #70% datos de entrenamiento, 30% prueba
                                                    stratify = y,
                                                    random_state = 1)
# Datos de entrenaminto
conteo = X_train.copy()
conteo['label'] = y_train
conteo.groupby('label').size()

In [ ]:
decision_tree = tree.DecisionTreeClassifier(criterion='gini',
                                             min_samples_split=20,
                                             min_samples_leaf=5,
                                             max_depth = depth,
                                             class_weight='balanced') 

# Ajustamos el modelo con los datos
decision_tree.fit(X_train, y_train)
y_pred_train = decision_tree.predict(X_train)

#Porcentaje de Exactitud con los datos de entrenamiento
acc_train = accuracy_score(y_train, y_pred_train)
print("Exactitud con datos de entrenamiento: {:.2f}".format(acc_train))

y_pred_test = decision_tree.predict(X_test)

#Porcentaje de Exactitud con pruebas
acc_test = accuracy_score(y_test, y_pred_test)
print("Exactitud con datos de pruebas: {:.2f}".format(acc_test))

## 4. ¿Qué variables usar? ¿Gráficos de correlación?
*Sug: Relacionar lo obtenido con los gráficos de correlación y SelectKBest, recuerden los costos computacionales de la técnica de árboles.*

###  Gráfico de Pearson: Correlación general de datos

In [ ]:
colormap = plt.cm.viridis
plt.figure(figsize=(10,10))
plt.title('Pearson Correlation of Features')
sb.heatmap(dataframe.astype(float).corr(),
           vmax=1.0,
           cmap=colormap,
           annot=True,
           linewidths=0.1,
           linecolor='white',
           square=True)

### Gráfico de Cramer V

In [ ]:
data_encoded = dataframe.copy()
from scipy.stats import chi2_contingency

def cramers_V(var1, var2):
  crosstab = np.array(pd.crosstab(var1, var2, rownames=None, colnames=None))
  stat = chi2_contingency(crosstab)[0]
  obs = np.sum(crosstab)
  mini = min(crosstab.shape) - 1
  return (stat/(obs*mini))

rows= []

for var1 in data_encoded:
  col = []
  for var2 in data_encoded :
    cramers =cramers_V(data_encoded[var1], data_encoded[var2])
    col.append(round(cramers,2))
  rows.append(col)
  
cramers_results = np.array(rows)
df = pd.DataFrame(cramers_results, columns = data_encoded.columns, index =data_encoded.columns)
with sb.axes_style("white"):
  ax = sb.heatmap(df,
                  vmin=0.,
                  vmax=1,
                  cmap=colormap,
                  annot=True,
                  linewidths=0.1,
                  square=True)

plt.show()

### SelectKBest

In [ ]:
#Todos los datos excepto la caracteristica objetivo.
X = dataframe.drop(['comprar'], axis=1)

#CARACTERÍSTICA OBJETIVO
y = dataframe['comprar']

best=SelectKBest(k=len(list(dataframe))-1)    #len(list(dataframe))-1: Máximo numero de atributos
                          #Para que el modelo no sea pesado, por el número de filas, deseo un número n=5, podrían ser más, hasta 12

X_new = best.fit_transform(X, y)
X_new.shape
selected = best.get_support(indices=True)
print(X.columns[selected])

Sino me siento seguro, de cuáles variables coger, se pueden realizar pruebas sobre si mejoran o no la exactitud de los árboles usando las variables que me sugieren los gráficos de correlación y SelectKbest

#### n=5 atributos

In [ ]:
#Todos los datos excepto la caracteristica objetivo.
X = dataframe.drop(['comprar'], axis=1)

#CARACTERÍSTICA OBJETIVO
y = dataframe['comprar']

best=SelectKBest(k=5)    #len(list(dataframe))-1: Máximo numero de atributos
                          #Para que el modelo no sea pesado, por el número de filas, deseo un número n=5, podrían ser más, hasta 12

X_new = best.fit_transform(X, y)
X_new.shape
selected = best.get_support(indices=True)
print(X.columns[selected])

In [ ]:
#Cuales atributos no están en la lista anterior
dataframe.columns

In [ ]:
#Todos los datos excepto la caracteristica comprar
X = dataframe.drop(['comprar','gastos_comunes','pago_coche','gastos_otros','vivienda','estado_civil'], axis=1)

#Atributo OBJETIVO
y = dataframe['comprar'] 

# Dividimos los datos para entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.1, #90% datos de entrenamiento, 10% prueba
                                                    stratify = y,
                                                    random_state = 1)
# Datos de entrenaminto
conteo = X_train.copy()
conteo['label'] = y_train
conteo.groupby('label').size()

In [ ]:
decision_tree = tree.DecisionTreeClassifier(criterion='gini',
                                             min_samples_split=20,
                                             min_samples_leaf=5,
                                             max_depth = depth,
                                             class_weight='balanced') 

# Ajustamos el modelo con los datos
decision_tree.fit(X_train, y_train)
y_pred_train = decision_tree.predict(X_train)

#Porcentaje de Exactitud con los datos de entrenamiento
acc_train = accuracy_score(y_train, y_pred_train)
print("Exactitud con datos de entrenamiento: {:.2f}".format(acc_train))

y_pred_test = decision_tree.predict(X_test)

#Porcentaje de Exactitud con pruebas
acc_test = accuracy_score(y_test, y_pred_test)
print("Exactitud con datos de pruebas: {:.2f}".format(acc_test))

#### n=6 atributos

In [ ]:
#Todos los datos excepto la caracteristica objetivo.
X = dataframe.drop(['comprar'], axis=1)

#CARACTERÍSTICA OBJETIVO
y = dataframe['comprar']

best=SelectKBest(k=6)    #len(list(dataframe))-1: Máximo numero de atributos
                          #Para que el modelo no sea pesado, por el número de filas, deseo un número n=6, podrían ser más, hasta 12

X_new = best.fit_transform(X, y)
X_new.shape
selected = best.get_support(indices=True)
print(X.columns[selected])

In [ ]:
#Cuales atributos no están en la lista anterior
dataframe.columns

In [ ]:
#Todos los datos excepto la caracteristica comprar
X = dataframe.drop(['comprar','pago_coche','gastos_otros','vivienda','estado_civil'], axis=1)

#Atributo OBJETIVO
y = dataframe['comprar'] 

# Dividimos los datos para entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.1, #90% datos de entrenamiento, 10% prueba
                                                    stratify = y,
                                                    random_state = 1)
# Datos de entrenaminto
conteo = X_train.copy()
conteo['label'] = y_train
conteo.groupby('label').size()

In [ ]:
decision_tree = tree.DecisionTreeClassifier(criterion='gini',
                                             min_samples_split=20,
                                             min_samples_leaf=5,
                                             max_depth = depth,
                                             class_weight='balanced') 

# Ajustamos el modelo con los datos
decision_tree.fit(X_train, y_train)
y_pred_train = decision_tree.predict(X_train)

#Porcentaje de Exactitud con los datos de entrenamiento
acc_train = accuracy_score(y_train, y_pred_train)
print("Exactitud con datos de entrenamiento: {:.2f}".format(acc_train))

y_pred_test = decision_tree.predict(X_test)

#Porcentaje de Exactitud con pruebas
acc_test = accuracy_score(y_test, y_pred_test)
print("Exactitud con datos de pruebas: {:.2f}".format(acc_test))

#### n=7 atributos

In [ ]:
#Todos los datos excepto la caracteristica objetivo.
X = dataframe.drop(['comprar'], axis=1)

#CARACTERÍSTICA OBJETIVO
y = dataframe['comprar']

best=SelectKBest(k=7)    #len(list(dataframe))-1: Máximo numero de atributos
                          #Para que el modelo no sea pesado, por el número de filas, deseo un número n=5, podrían ser más, hasta 12

X_new = best.fit_transform(X, y)
X_new.shape
selected = best.get_support(indices=True)
print(X.columns[selected])

In [ ]:
#Cuales atributos no están en la lista anterior
dataframe.columns

In [ ]:
#Todos los datos excepto la caracteristica comprar
X = dataframe.drop(['comprar','gastos_otros','vivienda','estado_civil'], axis=1)

#Atributo OBJETIVO
y = dataframe['comprar'] 

# Dividimos los datos para entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.1, #90% datos de entrenamiento, 10% prueba
                                                    stratify = y,
                                                    random_state = 1)
# Datos de entrenaminto
conteo = X_train.copy()
conteo['label'] = y_train
conteo.groupby('label').size()

In [ ]:
decision_tree = tree.DecisionTreeClassifier(criterion='gini',
                                             min_samples_split=20,
                                             min_samples_leaf=5,
                                             max_depth = depth,
                                             class_weight='balanced') 

# Ajustamos el modelo con los datos
decision_tree.fit(X_train, y_train)
y_pred_train = decision_tree.predict(X_train)

#Porcentaje de Exactitud con los datos de entrenamiento
acc_train = accuracy_score(y_train, y_pred_train)
print("Exactitud con datos de entrenamiento: {:.2f}".format(acc_train))

y_pred_test = decision_tree.predict(X_test)

#Porcentaje de Exactitud con pruebas
acc_test = accuracy_score(y_test, y_pred_test)
print("Exactitud con datos de pruebas: {:.2f}".format(acc_test))

## 5. Diferentes alturas del árbol

In [ ]:
#Luego del análisis anterior, me quedo con el conjunto de variables que deseo
df_def = dataframe.drop(['pago_coche','gastos_otros','vivienda','estado_civil'], axis=1)

In [ ]:
X = df_def.drop(['comprar'], axis=1)
y = df_def['comprar']

In [ ]:
#Para usar este método X y 'y' deben ser listas np
X=np.array(X)
y=np.array(y)

cv = StratifiedKFold(n_splits=6, shuffle=True, random_state=1) 
accuracies = list()
max_attributes = len(list(dataframe))
depth_range = range(1, max_attributes + 1)

# Testearemos la profundidad de 1 a cantidad de atributos +1
for depth in depth_range:
    fold_accuracy = []
    tree_model = tree.DecisionTreeClassifier(criterion='gini',
                                             min_samples_split=20,
                                             min_samples_leaf=5,
                                             max_depth = depth,
                                             class_weight='balanced') 
    for train_index, valid_index in cv.split(X, y):
        X_train, X_test = X[train_index], X[valid_index]
        y_train, y_test = y[train_index], y[valid_index]

        model = tree_model.fit(X_train, y_train)
        valid_acc = model.score(X_test, y_test) # calculamos la precision con el segmento de validacion
        fold_accuracy.append(valid_acc)

    # almacenamos acc promedio para cada profundidad
    avg = sum(fold_accuracy)/len(fold_accuracy)
    accuracies.append(avg)
    
# Mostramos los resultados obtenidos
df = pd.DataFrame({"Max Depth": depth_range, "Average Accuracy": accuracies})
df = df[["Max Depth", "Average Accuracy"]]
print(df.to_string(index=False))

#¿Dónde se alcanza el mejor valor de exactitud?

## 6. Mínimo de muestras por nodo, mínimo de muestra por hoja. ¿Fue necesario balancear las clases para los datos de entrenamiento?

In [ ]:
X = df_def.drop(['comprar'], axis=1)
y = df_def['comprar']

# Dividimos los datos para entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.1, #90% datos de entrenamiento, 10% prueba
                                                    stratify = y,
                                                    random_state = 1)

In [ ]:
conteo = X_train.copy()
conteo['label'] = y_train
conteo.groupby('label').size()

Se va a instanciar el árbol con una altura igual a 8

In [ ]:
# Instanciar Arbol de decision con profundidad = 5
decision_tree = tree.DecisionTreeClassifier(criterion='gini',
                                            #Restricciones que evitan Overfitting.
                                            min_samples_split=5, #5. Qué pasa si varió el número de muestras mínimo por nodo.
                                            min_samples_leaf=2,   #5. Qué pasa si vario el número de muestras mínimo por hoja (Las hojas es el nodo final del).
                                            max_depth = 8,    #4. Testear qué pasa si aumento o disminuyo la altura del arbol.
                                                              #Tener en cuenta que la altura va desde 1 hasta el número de atributos (Variables).
                                            class_weight='balanced')  #Balanceo automático
                                            #class_weight={0:1, 1:w})


In [ ]:
# Ajustamos el modelo con los datos
decision_tree.fit(X_train, y_train)

#Cada cambio que se haga requiere analizar o tener en cuenta su nivel de exactitud.

#Exactitud

#Datos de entrenamiento
y_pred_train = decision_tree.predict(X_train)

#Porcentaje de exactitud con los datos de entrenamiento
acc_train = accuracy_score(y_train, y_pred_train)
print("Exactitud con datos de entrenamiento: {:.2f}".format(acc_train))

#Datos de prueba
y_pred_test = decision_tree.predict(X_test)

#Porcentaje de exactitud con pruebas
acc_test = accuracy_score(y_test, y_pred_test)
print("Exactitud con datos de pruebas: {:.2f}".format(acc_test))  

In [ ]:
# exportar el modelo a archivo .dot y graficar
with open(r"tree1.dot", 'w') as f:
     f = tree.export_graphviz(decision_tree,
                              out_file=f,
                              max_depth = 8,
                              impurity = True,
                              feature_names = list(df_def.drop(['comprar'], axis=1)),
                              class_names = ['Alquilar', 'Comprar'],
                              rounded = True,
                              filled= True )
        
# Convertir el archivo .dot a png para poder visualizarlo
check_call(['dot','-Tpng',r'tree1.dot','-o',r'tree1.png'])
PImage("tree1.png")

## 7. Matriz de confunsión.
* tp (True positive): Valor que es verdad y que el modelo predijo correctamente.
* fn (False negative): Valor que es verdad y el modelo predijo incorrectamente.
* fp (False positive): Valor que no es verdad y que el modelo predijo incorrectamente.
* tn (True negative): Valor que no es verdad y que el modelo predijo correctamente.

### Entrenamiento

In [ ]:
disp = plot_confusion_matrix(decision_tree, X_train, y_train,
                                 display_labels=['Alquilar','Comprar'],
                                 cmap=plt.cm.Blues,
                                 normalize=None)
disp.ax_.set_title('Matriz de confusión del Árbol en Entrenamiento', y=1.02, size=18)

plt.show()

### Prueba.

In [ ]:
disp = plot_confusion_matrix(decision_tree, X_test, y_test,
                                 display_labels=['Alquilar','Comprar'],
                                 cmap=plt.cm.Blues,
                                 normalize=None)
disp.ax_.set_title('Matriz de confusión del Árbol en Prueba', y=1.02, size=18)

plt.show()

## 8. Métrica de exactitud

### Entrenamiento.

In [ ]:
y_pred_train = decision_tree.predict(X_train)
acc_train = accuracy_score(y_train, y_pred_train)
print("Train set accuracy: {:.2f}".format(acc_train))

### Prueba.

In [ ]:
y_pred_test = decision_tree.predict(X_test)
acc_test = accuracy_score(y_test, y_pred_test)
print("Test set accuracy: {:.2f}".format(acc_test))

## 9. F1-Score.

### Entrenamiento.

In [ ]:
decoded_y_test = y_train
decoded_predictions = decision_tree.predict(X_train)
print(f'Reporte de clasificación:')
print(classification_report(decoded_y_test,
                            decoded_predictions))

### Prueba.

In [ ]:
decoded_y_test = y_test
decoded_predictions = decision_tree.predict(X_test)
print(f'Reporte de clasificación:')
print(classification_report(decoded_y_test,
                            decoded_predictions))

## 10. Clasificación bayesiana.

In [ ]:
#Ajustando el modelo
# Instantiate the classifier
gnb = GaussianNB()
# Train classifier
gnb.fit(
    X_train,
    y_train
)
y_pred = gnb.predict(X_test)
 
print('Accuracy in training set: {:.2f}'
     .format(gnb.score(X_train, y_train)))
print('Accuracy in test set: {:.2f}'
     .format(gnb.score(X_test, y_test)))


### Matriz de confusión.

#### Entrenamiento.

In [ ]:
disp = plot_confusion_matrix(gnb, X_train, y_train,
                                 display_labels=['Alquilar','Comprar'],
                                 cmap=plt.cm.Blues,
                                 normalize=None)
disp.ax_.set_title('Matriz de confusión del clasificador bayesiano en Entrenamiento', y=1.02, size=18)

plt.show()

#### Prueba.

In [ ]:
disp = plot_confusion_matrix(gnb, X_test, y_test,
                                 display_labels=['Alquilar','Comprar'],
                                 cmap=plt.cm.Blues,
                                 normalize=None)
disp.ax_.set_title('Matriz de confusión del clasificador bayesiano en Prueba', y=1.02, size=18)

plt.show()

### F1-Score

#### Entrenamiento.

In [ ]:
decoded_y_test = y_train
decoded_predictions = gnb.predict(X_train)
print(f'Reporte de clasificación:')
print(classification_report(decoded_y_test,
                            decoded_predictions))

#### Prueba.

In [ ]:
decoded_y_test = y_test
decoded_predictions = gnb.predict(X_test)
print(f'Reporte de clasificación:')
print(classification_report(decoded_y_test,
                            decoded_predictions))

## 11. Análisis de resultados.

### Clasificador

In [ ]:
#¿Qué variables/atributos tengo?
df_def.head()

In [ ]:
#Predicción con ambos modelos
data = {'ingresos':[2000,6000,2000],
        'gastos_comunes':[1360,920,1200],
        'ahorros':[5000,34000,10000],
        'hijos':[0,2,1],
        'trabajo':[5,5,8],
        'gastos':[2500,2000,1000],
        'financiar':[200000,320000,150000]}
prueba = pd.DataFrame(data)

In [ ]:
print(gnb.predict(prueba))

In [ ]:
#¿Qué tan seguro es la predicción del clasificador?
x_test=pd.DataFrame(columns=('comprar','ingresos','gastos_comunes','ahorros','hijos','trabajo','gastos','financiar'))
x_test.loc[0]=(1000,2000,1360,5000,0,5,2500,200000)    #El primer dato es de comprar y no importa, ya que ese valor es el que se va a predecir
y_pred = gnb.predict(x_test.drop(['comprar'], axis = 1)) #Saca 'comprar':  Característica obj
print("Prediccion: " + str(y_pred))
y_proba = gnb.predict_proba(x_test.drop(['comprar'], axis = 1))
print("Probabilidad de Acierto: " + str(np.round(y_proba[0][y_pred]* 100, 2)) + "%")     

In [ ]:
x_test.loc[0]=(1000,6000,920,34000,2,5,2000,320000)    #El primer dato es de comprar y no importa, ya que ese valor es el que se va a predecir
y_pred = gnb.predict(x_test.drop(['comprar'], axis = 1)) #Saca 'comprar':  Característica obj
print("Prediccion: " + str(y_pred))
y_proba = gnb.predict_proba(x_test.drop(['comprar'], axis = 1))
print("Probabilidad de Acierto: " + str(np.round(y_proba[0][y_pred]* 100, 2)) + "%")     

### Árbol

In [ ]:
x_test.loc[0]=(1000,2000,1360,5000,0,5,2500,200000)    #El primer dato es de comprar y no importa, ya que ese valor es el que se va a predecir
y_pred = decision_tree.predict(x_test.drop(['comprar'], axis = 1)) #Saca 'comprar':  Característica obj
print("Prediccion: " + str(y_pred))
y_proba = decision_tree.predict_proba(x_test.drop(['comprar'], axis = 1))
print("Probabilidad de Acierto: " + str(np.round(y_proba[0][y_pred]* 100, 2)) + "%")    

In [ ]:
x_test.loc[0]=(1000,6000,920,34000,2,5,2000,320000)    #El primer dato es de comprar y no importa, ya que ese valor es el que se va a predecir
y_pred = decision_tree.predict(x_test.drop(['comprar'], axis = 1)) #Saca 'comprar':  Característica obj
print("Prediccion: " + str(y_pred))
y_proba = decision_tree.predict_proba(x_test.drop(['comprar'], axis = 1))
print("Probabilidad de Acierto: " + str(np.round(y_proba[0][y_pred]* 100, 2)) + "%")     